# Spot enrichment tools


We need to go through all of the clustering steps before we can run any of these commands. We'll import a script that does all of this for us. If you'd like more information you can find it [here].

In [ ]:
source("scripts//visium_obj.R")

Now we'll import our signature matrix information so that we can make matrices for the following methods:

In [ ]:
sign_matrix_path = system.file("extdata", "sig_matrix.txt", package = 'Giotto')
brain_sc_markers = data.table::fread(sign_matrix_path)
sig_matrix = as.matrix(brain_sc_markers[,-1]); rownames(sig_matrix) = brain_sc_markers$Event
final_sig = sig_matrix[, 1:4]

Below we will create variables that hold all the cell types and genes of interest:

In [ ]:
sigmatrix <- as.data.frame(final_sig)
cell_types <- colnames(final_sig)
sign_genes <- list()
for (i in 1:length(colnames(final_sig))) {
  sign_genes[[length(sign_genes)+1]] <- row.names(sigmatrix)[which(sigmatrix[i]==1)]
}

### 1. Parametric analysis of gene set enrichment (PAGE)


Now we can make a [signature matrix](https://rubd.github.io/Giotto_site/reference/makeSignMatrixPAGE.html) for PAGE:

In [ ]:
astro_epen_markers = c("Krt15" , "Apoc1" , "Igsf1" , "Gjb6" , "Slc26a3" ,
                       "1500015O10Rik" , "S1pr1" , "Riiad1" , "Cldn10" , "Itih3" ,
                       "Ccdc153" , "Cbs" , "C4b" , "Gm11627" , "Folr1" ,
                       "Calml4" , "Aqp4" , "Ppp1r3g" , "1700012B09Rik" , "Hes5")

gran_markers = c("Nr3c2", "Gabra5", "Tubgcp2", "Ahcyl2",
                 "Islr2", "Rasl10a", "Tmem114", "Bhlhe22", 
                 "Ntf3", "C1ql2")

cortex_hippo_markers = c("6330403A02Rik" , "Tekt5" , "Wipf3" , "1110032F04Rik" , "Lmo3" ,
                         "Nrep" , "Slc30a3" , "Plcxd2" , "D630023F18Rik" , "Nptx1" ,
                         "C1ql3" , "Ddit4l" , "Fezf2" , "Col24a1" , "Kcnf1" ,
                         "Tnnc1" , "Gm12371" , "3110035E14Rik" , "Nr4a2" , "Nr4a3")

oligo_markers = c("Efhd1" , "H2-Ab1" , "Enpp6" , "Ninj2" , "Bmp4" ,
                  "Tnr" , "Hapln2" , "Neu4" , "Wfdc18" , "Ccp110" ,
                  "Gm26834" , "Il23a" , "Arap2" , "Nkx2-9" , "Mal" ,
                  "Tmem2" , "Birc2" , "Cdkn1c" , "Pak4" , "Tmem88b")

In [ ]:
signature_matrix = makeSignMatrixPAGE(sign_names = cell_types,
                                      sign_list = sign_genes)

And finally, we can run our [PAGE](https://rubd.github.io/Giotto_site/reference/runPAGEEnrich.html)

In [ ]:
# runSpatialEnrich() can also be used as a wrapper for all currently provided enrichment options
my_visium_object = runPAGEEnrich(gobject = my_visium_object,
                                 sign_matrix = signature_matrix,
                                 min_overlap_genes = 2)

Now let's visualize:

In [ ]:
cell_types_subset = colnames(signature_matrix)
spatCellPlot(gobject = my_visium_object, 
             spat_enr_names = 'PAGE',
             cell_annotation_values = cell_types_subset,
             cow_n_col = 2,coord_fix_ratio = NULL, point_size = 2.75)[0]

### 2. RANK enrichment


First, we'll read in our single cell and annotations data (this can be found in the ```data``` folder in this directory:

In [ ]:
# single cell matrix
single_cell_DT = fread("data/raw_exp_small.txt")
single_cell_matrix = Giotto:::dt_to_matrix(single_cell_DT)

# single cell annotation vector
cell_annotations = read.table(file = "data/major_cluster_small.txt")
cell_annotations = as.vector(cell_annotations$V1)

Now we can create our RANK matrix:

In [ ]:
rank_matrix = makeSignMatrixRank(sc_matrix = single_cell_matrix, sc_cluster_ids = cell_annotations)

Run RANK enrichment:

In [ ]:
my_visium_object = runRankEnrich(gobject = my_visium_object, sign_matrix = rank_matrix)

And visualize:

In [ ]:
cell_types_subset = c("Astro_ependymal", "Oligo_dendrocyte" , "Cortex_hippocampus" , "Granule_neurons" )
spatCellPlot(gobject = my_visium_object, 
             spat_enr_names = 'rank',
             cell_annotation_values = cell_types_subset,
             cow_n_col = 2,coord_fix_ratio = NULL, point_size = 1.75)[0]

### 3. Hypergeometric enrichment
---
First we'll run our [hypergeometric enrichment](https://rubd.github.io/Giotto_site/reference/runHyperGeometricEnrich.html):

In [ ]:
my_visium_object = runHyperGeometricEnrich(gobject = my_visium_object,
                                        sign_matrix = signature_matrix)

Now we can visualize out results:

In [ ]:
cell_types_subset = colnames(signature_matrix)

spatCellPlot(gobject = my_visium_object, 
             spat_enr_names = 'hypergeometric',
             cell_annotation_values = cell_types_subset,
             cow_n_col = 2,coord_fix_ratio = NULL, point_size = 2.75)[0]
             

### 4. DWLS Deconvolution

In [ ]:
dwls_matrix = makeSignMatrixDWLS(gobject = my_visium_object,
                                 sign_gene = unlist(sign_genes),
                                 cell_type = cell_types)

In [ ]:
my_visium_object = runDWLSDeconv(gobject = my_visium_object,
                                 sign_matrix = dwls_matrix)

In [ ]:
plot_data <- as.data.frame(my_visium_object@spatial_enrichment$DWLS)[-1]
plot_col <- colnames(plot_data)
plot_data$x <- as.numeric(as.character(my_visium_object@spatial_locs$sdimx))
plot_data$y <- as.numeric(as.character(my_visium_object@spatial_locs$sdimy))
min_x <- min(plot_data$x)
plot_data$radius <- 50
df <- data.frame()
p <- ggplot(df) + geom_point() + xlim(min(plot_data$x)-1, max(plot_data$x)+1) + ylim(min(plot_data$y)-1, max(plot_data$y)+1)
p + geom_scatterpie(aes(x=x, y=y, r=radius), data=plot_data, cols=plot_col, color=NA, alpha=.8) +
  geom_scatterpie_legend(plot_data$radius, x=1, y=1) + theme_classic()